# 1.0 - IMPORTS

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

# 2.0 - LOADING DATA

In [40]:
# Carregando o arquivo CSV usando caminho relativo
df = pd.read_csv('dataset/data.csv', encoding='latin1')

In [41]:
# Verificando as primeiras linhas para confirmar a leitura
df.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


# 3.0 - DATA DESCRIPTION

In [42]:
# checando os tipos de dados de cada coluna
df.dtypes

InvoiceNo       object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID     float64
Country         object
dtype: object

In [43]:
# mostrando a quantidade de linhas e colunas do dataframe
df.shape

(541909, 8)

In [45]:
# verificando a existência de valores ausentes e a proporção de dados faltantes
missing_values = df.isnull().sum()
missing_percentage = (missing_values / df.shape[0]) * 100
missing_df = pd.DataFrame({'Total de Valores Faltantes': missing_values, 'Porcentagem': missing_percentage})
missing_df

,Total de Valores Faltantes,Porcentagem
InvoiceNo,0,0.000000
StockCode,0,0.000000
Description,1454,0.268311
Quantity,0,0.000000
InvoiceDate,0,0.000000
UnitPrice,0,0.000000
CustomerID,135080,24.926694
Country,0,0.000000
